In [32]:
def count_permutations(n):
    p = list(range(1, n + 1))
    permutations = [p.copy()]
    
    while True:
        j = None
        for i in range(n - 1, 0, -1):
            if p[i - 1] < p[i]:
                j = i - 1
                break
        if j is None:
            break
        for i in range(n - 1, 0, -1):
            if p[i] > p[j]:
                k = i
                break
        p[j], p[k] = p[k], p[j]
        p[j+1:] = p[:j:-1]
        
        permutations.append(p.copy())
        
    return permutations
        
print(count_permutations(4))
            
            
        
        
    
            

[[1, 2, 3, 4], [1, 2, 4, 3], [1, 3, 2, 4], [1, 3, 4, 2], [1, 4, 2, 3], [1, 4, 3, 2], [2, 1, 3, 4], [2, 1, 4, 3], [2, 3, 1, 4], [2, 3, 4, 1], [2, 4, 1, 3], [2, 4, 3, 1], [3, 1, 2, 4], [3, 1, 4, 2], [3, 2, 1, 4], [3, 2, 4, 1], [3, 4, 1, 2], [3, 4, 2, 1], [4, 1, 2, 3], [4, 1, 3, 2], [4, 2, 1, 3], [4, 2, 3, 1], [4, 3, 1, 2], [4, 3, 2, 1]]


In [46]:
import random as rnd
import re
    

def task_rsa():
    with open('russian_nouns.txt', mode='r', encoding='utf8') as f:
        words = [word.strip() for word in f]

    words4 = [word for word in words if len(word) == 4]
    primes = [5, 7, 11, 13, 17]
    
    while True:
        rnd.shuffle(primes)
        p = primes[0]
        q = primes[1]
        n = p * q
        phi = euler_phi(n)
        e = rnd.randint(2, phi - 2)
        if gcd(e, phi) != 1:
            continue
        R_phi = IntegerModRing(phi)
        d = R_phi(1) / R_phi(e)
        
        random_word = words4[rnd.randint(0, len(words4) - 1)]
        codes = [ord(letter) - ord('а') + 2 for letter in random_word]
        encoded_codes = [code ^ e % n for code in codes]
        
        def have_similar_codes(list1, list2):
            for a, b in zip(list1, list2):
                if a == b:
                    return True
            return False
    
        
        if have_similar_codes(codes, encoded_codes):
            continue
            
        codes_text = ','.join(str(code) for code in encoded_codes)
        
        yield f'''Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
        на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
        от других букв.
        Шифрованое сообщение: $({codes_text})$, открытый ключ $({n}, {e})$. Восстановите закрытый ключ и
        расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.'''
        
def task_haffman(symbols_count=6):
    with open('russian_nouns.txt', mode='r', encoding='utf8') as f:
        words = [word.strip() for word in f]

    words = [word for word in words if len(word) == symbols_count and len(set(word)) == symbols_count]
    
    while True:
        random_word = words[rnd.randint(0, len(words) - 1)]
        random_frequencies = rnd.choices(range(100), k=symbols_count)
        
        symbols = sorted(zip(random_frequencies, random_word))
        process = symbols.copy()
        wrong_counts = False
        for _ in range(symbols_count - 2):
            c1, s1 = process.pop(0)
            c2, s2 = process.pop(0)
            c3, _ = process[0]
            if c1 == c2 or c2 == c3:
                wrong_counts = True
                break
            process.insert(0, (c1 + c2, s1 + s2))
            process.sort()
        if wrong_counts:
            continue
        if len(process[0][1]) == 1 or len(process[1][1]) == 1:
            continue
            
        frequencies_texts = ', '.join(f'`{s}`: {c}' for c, s in symbols)
            
        yield f'''Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
        {frequencies_texts}. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.'''
        
        
def task_permutation_by_number(n=7):
    all_permutations = count_permutations(n)
    while True:
        number = rnd.randint(42, factorial(n) - 42)
        yield f'''Если упорядочить все перестановки чисел от 1 до {n}, какая перестановка идет под номером {number}? (Ответ {all_permutations[number - 1]})'''
        
def task_permutation_number(n=7):
    numbers = list(range(1, n + 1))
    
    while True:
        rnd.shuffle(numbers)
        if numbers[0] == n or numbers[0] == 1:
            continue
        permutation = ''.join(str(a) for a in numbers)
        all_permutations = count_permutations(n)
        ans = all_permutations.index(numbers)
        yield f'''Если упорядочить все перестановки чисел от 1 до {n}, какой по счету будет перестановка {permutation}? (Ответ {ans + 1})'''
    
    
def task_polygcd(deg=4, gcd_deg=2):
    x = var('x')
    poly_ring = QQ[x]
    coefs_variants = [-3, -2, -1, 1, 2, 3]
    
    while True:
        gcd_coefs = rnd.choices(coefs_variants, k=gcd_deg + 1)
        mul1_coefs = rnd.choices(coefs_variants, k=deg - gcd_deg + 1)
        mul2_coefs = rnd.choices(coefs_variants, k=deg - gcd_deg + 1)
        def coefs_to_poly(coefs):
            return poly_ring(sum(coef * x^i for i, coef in enumerate(coefs)))

        gcd_poly = coefs_to_poly(gcd_coefs)
        mul1_poly = coefs_to_poly(mul1_coefs)
        mul2_poly = coefs_to_poly(mul2_coefs)

        p1 = gcd_poly * mul1_poly
        p2 = gcd_poly * mul2_poly

        g = gcd(p1, p2)
        if g.degree() > gcd_deg:
            continue

        yield fr'''Вычислите НОД многочленов ${latex(p1)}$ и ${latex(p2)}$ над $\mathbf{{R}}[x]$. (Ответ: ${latex(g)}$)'''
        
            
def task_polygcd_mod(deg=4, gcd_deg=2, mods=(3, 5, 7)):
    x = var('x')
    mod = rnd.choice(mods)
    poly_ring = IntegerModRing(mod)[x]
    coefs_variants = list(range(1, mod))
    
    #TODO dry
    while True:
        gcd_coefs = rnd.choices(coefs_variants, k=gcd_deg + 1)
        mul1_coefs = rnd.choices(coefs_variants, k=deg - gcd_deg + 1)
        mul2_coefs = rnd.choices(coefs_variants, k=deg - gcd_deg + 1)
        def coefs_to_poly(coefs):
            return poly_ring(sum(coef * x^i for i, coef in enumerate(coefs)))

        gcd_poly = coefs_to_poly(gcd_coefs)
        mul1_poly = coefs_to_poly(mul1_coefs)
        mul2_poly = coefs_to_poly(mul2_coefs)

        p1 = gcd_poly * mul1_poly
        p2 = gcd_poly * mul2_poly

        g = gcd(p1, p2)
        if g.degree() > gcd_deg:
            continue

        yield f'''Вычислите НОД многочленов ${latex(p1)}$ и ${latex(p2)}$ над $\\mathbf{{Z}}_{mod}[x]$. (Ответ ${latex(g)}$)'''
    
def task_include_exclude(divs_count=4):
    while True:
        n = rnd.randint(1000, 9999)
        divs = sorted(rnd.choices(range(2, 20), k=divs_count))
        wrong_division = False
        for i in range(divs_count):
            for j in range(i + 1, divs_count):
                if divs[j] % divs[i] == 0:
                    wrong_division = True
        if wrong_division:
            continue    
            
        divs_text = ', '.join(str(div) for div in divs)
        
        ans = 0
        for x in range(1, n + 1):
            for d in divs:
                if x % d == 0:
                    ans += 1
                    break;

        yield f'''Сколько чисел от $1$ до ${n}$ делится хотя бы на одно из этих чисел: ${{{divs_text}}}$. (Ответ {ans})?'''

In [47]:
from IPython.display import Markdown, display


def pad_text(text, level=1):
    pad = "  " * level
    lines = [pad + line.strip() for line in text.split('\n')]
    return '\n'.join(lines)


def generate_tasks(tasks, variants_number, seed=None, seeds=None):
    if seeds is None:
        if seed is None:
            seeds = rnd.choices(range(2**63 - 1), k=len(tasks))
        else:
            seeds = [seed] * len(tasks)
            
    tasks_states = []
    for seed in seeds:
        if isinstance(seed, Integer):
            seed = int(seed)
        rnd.seed(seed)
        tasks_states.append(rnd.getstate())
    
    md = []
    for variant in range(1, variants_number + 1):
        md.append(f'\n## Вариант {variant}')
        
        next_tasks_states = []
        
        for task, state in zip(tasks, tasks_states):
            rnd.setstate(state)
            task_text = '1. ' + pad_text(next(task))
            md.append(task_text)
            next_tasks_states.append(rnd.getstate())
            
        tasks_states = next_tasks_states
        
    return Markdown('\n'.join(md))

In [48]:
text = generate_tasks([
    task_rsa(),
    task_haffman(4),
    task_permutation_by_number(),
    task_permutation_number(),
    task_polygcd(),
    task_polygcd_mod(),
    task_include_exclude()
], 30, seed=123)
display(text)


## Вариант 1
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(172,168,152,172)$, открытый ключ $(187, 89)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `с`: 29, `м`: 43, `з`: 46, `а`: 47. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 470? (Ответ [1, 5, 7, 4, 2, 6, 3])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6524731? (Ответ 4122)
1.   Вычислите НОД многочленов $3 x^{4} + 6 x^{3} + 3 x^{2} + 9$ и $-3 x^{4} - 7 x^{3} - 4 x^{2} + 3 x - 3$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + 3 x + 3$)
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x^{2} + 2$ и $x^{4} + 2 x^{2} + 2 x + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + 2 x + 1$)
1.   Сколько чисел от $1$ до $9400$ делится хотя бы на одно из этих чисел: ${9, 11, 12, 16}$. (Ответ 2634)?

## Вариант 2
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(94,24,67,94)$, открытый ключ $(119, 23)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `с`: 43, `в`: 53, `з`: 59, `о`: 79. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 2234? (Ответ [4, 1, 6, 2, 3, 7, 5])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6724513? (Ответ 4241)
1.   Вычислите НОД многочленов $x^{4} + 4 x^{3} + 8 x^{2} + 11 x + 6$ и $3 x^{4} + 2 x^{3} + 11 x^{2} + 9$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + x + 3$)
1.   Вычислите НОД многочленов $2 x^{4} + x^{2} + x + 1$ и $x^{4} + 2 x^{3} + x^{2} + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 2$)
1.   Сколько чисел от $1$ до $6765$ делится хотя бы на одно из этих чисел: ${9, 11, 16, 19}$. (Ответ 1910)?

## Вариант 3
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(15,131,32,42)$, открытый ключ $(221, 101)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `м`: 41, `а`: 51, `с`: 52, `з`: 84. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 756? (Ответ [2, 1, 4, 5, 7, 6, 3])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 5321467? (Ответ 3145)
1.   Вычислите НОД многочленов $6 x^{4} - 11 x^{3} - 10 x^{2} + 9 x + 6$ и $3 x^{4} + 8 x^{3} - 14 x^{2} - 3 x + 6$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - \frac{1}{3} x - \frac{2}{3}$)
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{2} + 2$ и $2 x^{4} + x^{3} + x + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 1$)
1.   Сколько чисел от $1$ до $5200$ делится хотя бы на одно из этих чисел: ${13, 14, 16, 19}$. (Ответ 1220)?

## Вариант 4
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(40,25,9,16)$, открытый ключ $(77, 49)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `з`: 62, `с`: 63, `а`: 96, `м`: 99. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 3378? (Ответ [5, 6, 1, 4, 7, 3, 2])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 3461275? (Ответ 1754)
1.   Вычислите НОД многочленов $-4 x^{4} - 2 x^{3} + 8 x^{2} - 7 x + 2$ и $4 x^{4} + 8 x^{3} + x^{2} + x - 2$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + \frac{3}{2} x - 1$)
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + 2 x^{2} + x + 1$ и $x^{4} + 2 x^{3} + x^{2} + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 2$)
1.   Сколько чисел от $1$ до $7865$ делится хотя бы на одно из этих чисел: ${3, 4, 14, 17}$. (Ответ 4339)?

## Вариант 5
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(28,97,48,28)$, открытый ключ $(143, 113)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `з`: 63, `в`: 68, `с`: 79, `о`: 95. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 2225? (Ответ [4, 1, 5, 6, 7, 2, 3])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6152473? (Ответ 3676)
1.   Вычислите НОД многочленов $2 x^{4} + 3 x^{3} - 2 x^{2} - 5 x - 2$ и $-2 x^{4} + 7 x^{3} + x^{2} - 7 x - 2$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - \frac{1}{2} x - 1$)
1.   Вычислите НОД многочленов $2 x^{4} + 2$ и $2 x^{4} + x^{3} + 2 x^{2} + 1$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 2$)
1.   Сколько чисел от $1$ до $6016$ делится хотя бы на одно из этих чисел: ${4, 13, 15, 18}$. (Ответ 2253)?

## Вариант 6
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(68,71,58,68)$, открытый ключ $(77, 7)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `а`: 37, `з`: 54, `м`: 79, `с`: 89. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 924? (Ответ [2, 3, 6, 4, 7, 5, 1])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 3675412? (Ответ 2039)
1.   Вычислите НОД многочленов $-6 x^{4} + 4 x^{3} - 6 x + 4$ и $6 x^{4} - x^{3} - 14 x^{2} - x + 6$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + \frac{1}{3} x - \frac{2}{3}$)
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{2} + 2$ и $2 x^{4} + x^{2} + 2 x + 1$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 1$)
1.   Сколько чисел от $1$ до $1337$ делится хотя бы на одно из этих чисел: ${8, 15, 17, 19}$. (Ответ 363)?

## Вариант 7
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(49,39,32,59)$, открытый ключ $(85, 61)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `з`: 38, `м`: 62, `а`: 75, `с`: 82. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 354? (Ответ [1, 4, 7, 5, 6, 3, 2])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2671453? (Ответ 1300)
1.   Вычислите НОД многочленов $3 x^{4} - 2 x^{3} + 3 x^{2} + 1$ и $-3 x^{4} + 8 x^{3} + 8 x^{2} + 5 x + 2$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + \frac{1}{3} x + \frac{1}{3}$)
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{2} + 2$ и $2 x^{4} + x^{3} + x + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 1$)
1.   Сколько чисел от $1$ до $2316$ делится хотя бы на одно из этих чисел: ${11, 15, 18, 19}$. (Ответ 541)?

## Вариант 8
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(66,3,67,66)$, открытый ключ $(119, 5)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `с`: 34, `а`: 74, `м`: 87, `з`: 98. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 3147? (Ответ [5, 3, 2, 1, 6, 4, 7])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6237154? (Ответ 3764)
1.   Вычислите НОД многочленов $-6 x^{4} - 7 x^{3} - 13 x^{2} - 7 x - 3$ и $-6 x^{4} - 9 x^{3} - 14 x^{2} - 10 x - 3$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + \frac{1}{2} x + \frac{3}{2}$)
1.   Вычислите НОД многочленов $2 x^{4} + 2$ и $x^{4} + 2 x^{3} + x^{2} + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 2$)
1.   Сколько чисел от $1$ до $8627$ делится хотя бы на одно из этих чисел: ${7, 12, 13, 19}$. (Ответ 2699)?

## Вариант 9
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(117,41,152,117)$, открытый ключ $(221, 59)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `м`: 13, `а`: 50, `с`: 51, `з`: 54. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 4434? (Ответ [7, 1, 6, 4, 5, 3, 2])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2476513? (Ответ 1079)
1.   Вычислите НОД многочленов $x^{4} - x^{3} - 5 x^{2} + 7 x - 2$ и $3 x^{4} + 9 x^{3} + x^{2} - 7 x + 2$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + 2 x - 1$)
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + 2 x^{2} + x + 1$ и $x^{4} + 1$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 2$)
1.   Сколько чисел от $1$ до $5356$ делится хотя бы на одно из этих чисел: ${8, 12, 17, 19}$. (Ответ 1376)?

## Вариант 10
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(31,25,26,31)$, открытый ключ $(55, 23)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `м`: 50, `а`: 75, `з`: 89, `с`: 95. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 4648? (Ответ [7, 3, 5, 4, 2, 6, 1])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 4132756? (Ответ 2189)
1.   Вычислите НОД многочленов $x^{4} + 3 x^{3} + 6 x^{2} + 6 x + 4$ и $x^{4} + 5 x^{3} + 11 x^{2} + 12 x + 6$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + 2 x + 2$)
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + x^{2} + 2$ и $x^{4} + 1$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 2$)
1.   Сколько чисел от $1$ до $3281$ делится хотя бы на одно из этих чисел: ${3, 4, 10, 14}$. (Ответ 1812)?

## Вариант 11
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(61,53,37,58)$, открытый ключ $(77, 13)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `в`: 48, `о`: 73, `з`: 77, `с`: 78. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 2763? (Ответ [4, 7, 1, 2, 5, 3, 6])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6217534? (Ответ 3743)
1.   Вычислите НОД многочленов $-3 x^{4} + 5 x^{3} - 3 x + 1$ и $-2 x^{4} + 5 x^{3} - 7 x^{2} + 7 x - 3$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - 2 x + 1$)
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{3} + 2 x + 2$ и $x^{4} + x^{3} + x^{2} + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + 2 x + 1$)
1.   Сколько чисел от $1$ до $4495$ делится хотя бы на одно из этих чисел: ${2, 9, 11, 17}$. (Ответ 2784)?

## Вариант 12
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(39,20,26,39)$, открытый ключ $(55, 33)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `з`: 34, `с`: 71, `в`: 82, `о`: 86. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 2833? (Ответ [4, 7, 5, 1, 2, 3, 6])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 5164237? (Ответ 2965)
1.   Вычислите НОД многочленов $6 x^{4} - 3 x^{3} - 3 x^{2} + 9 x - 6$ и $6 x^{4} + x^{3} - 9 x^{2} + x + 2$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + \frac{1}{2} x - 1$)
1.   Вычислите НОД многочленов $2 x^{4} + x^{2} + x + 1$ и $2 x^{4} + 2 x^{3} + 2 x^{2} + 1$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + 2 x + 1$)
1.   Сколько чисел от $1$ до $9898$ делится хотя бы на одно из этих чисел: ${6, 8, 9, 19}$. (Ответ 3385)?

## Вариант 13
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(117,56,100,93)$, открытый ключ $(119, 61)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `с`: 34, `з`: 36, `м`: 58, `а`: 69. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 467? (Ответ [1, 5, 7, 3, 6, 2, 4])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 4531276? (Ответ 2570)
1.   Вычислите НОД многочленов $6 x^{4} + 3 x^{3} - 8 x^{2} - 6 x - 1$ и $4 x^{4} + 8 x^{3} + 7 x^{2} + 4 x + 1$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + \frac{3}{2} x + \frac{1}{2}$)
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{3} + 2 x + 2$ и $2 x^{4} + 2 x^{3} + 2 x^{2} + 1$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + 2 x + 1$)
1.   Сколько чисел от $1$ до $6869$ делится хотя бы на одно из этих чисел: ${9, 10, 11, 13}$. (Ответ 2256)?

## Вариант 14
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(59,30,67,32)$, открытый ключ $(119, 83)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `з`: 35, `с`: 36, `а`: 42, `м`: 66. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 1349? (Ответ [2, 7, 3, 1, 6, 4, 5])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 5642137? (Ответ 3439)
1.   Вычислите НОД многочленов $-9 x^{4} - 6 x^{3} - 6 x^{2} - 5 x - 2$ и $-6 x^{4} - 15 x^{3} - 17 x^{2} - 9 x - 2$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + x + \frac{1}{3}$)
1.   Вычислите НОД многочленов $2 x^{4} + x^{3} + 2 x^{2} + 1$ и $2 x^{4} + 2 x^{2} + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 1$)
1.   Сколько чисел от $1$ до $2955$ делится хотя бы на одно из этих чисел: ${4, 7, 13, 18}$. (Ответ 1267)?

## Вариант 15
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(178,115,135,15)$, открытый ключ $(187, 147)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `о`: 32, `з`: 36, `с`: 47, `в`: 54. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 1148? (Ответ [2, 5, 4, 7, 1, 6, 3])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6324751? (Ответ 3876)
1.   Вычислите НОД многочленов $-3 x^{4} - 7 x^{3} + x^{2} - 4 x - 2$ и $-3 x^{4} - x^{3} + 9 x^{2} - 8 x + 6$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - \frac{2}{3} x + \frac{2}{3}$)
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x + 1$ и $2 x^{4} + 2 x^{2} + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + 2 x + 1$)
1.   Сколько чисел от $1$ до $1877$ делится хотя бы на одно из этих чисел: ${5, 7, 11, 13}$. (Ответ 796)?

## Вариант 16
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(91,71,42,91)$, открытый ключ $(143, 19)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `з`: 34, `о`: 60, `с`: 84, `в`: 87. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 2804? (Ответ [4, 7, 2, 6, 1, 5, 3])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2375146? (Ответ 949)
1.   Вычислите НОД многочленов $2 x^{4} - 7 x^{3} + 7 x^{2} - 7 x + 2$ и $-3 x^{4} + 10 x^{3} - 4 x^{2} - 5 x + 2$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - 3 x + 1$)
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + x^{2} + 2$ и $2 x^{4} + x^{2} + 2 x + 1$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 1$)
1.   Сколько чисел от $1$ до $4542$ делится хотя бы на одно из этих чисел: ${2, 3, 7, 11}$. (Ответ 3362)?

## Вариант 17
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(76,27,59,185)$, открытый ключ $(221, 107)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `м`: 28, `с`: 77, `а`: 89, `з`: 98. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 4637? (Ответ [7, 3, 5, 1, 6, 2, 4])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 4732615? (Ответ 2819)
1.   Вычислите НОД многочленов $4 x^{4} - 2 x^{3} - 6 x^{2} - 15 x - 9$ и $4 x^{4} + 10 x^{3} + 10 x^{2} + 7 x - 3$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + x + \frac{3}{2}$)
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + x^{2} + 2$ и $x^{4} + 2 x^{2} + x + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 1$)
1.   Сколько чисел от $1$ до $1511$ делится хотя бы на одно из этих чисел: ${2, 5, 11, 17}$. (Ответ 993)?

## Вариант 18
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(40,36,9,40)$, открытый ключ $(77, 49)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `з`: 61, `м`: 79, `с`: 83, `а`: 88. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 2776? (Ответ [4, 7, 1, 5, 3, 6, 2])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6452317? (Ответ 4041)
1.   Вычислите НОД многочленов $6 x^{4} + 5 x^{3} + 5 x^{2} + x - 2$ и $6 x^{4} + 8 x^{3} - 3 x^{2} - 3 x + 1$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + \frac{1}{3} x - \frac{1}{3}$)
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x^{2} + 2$ и $2 x^{4} + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + 2 x + 2$)
1.   Сколько чисел от $1$ до $5656$ делится хотя бы на одно из этих чисел: ${13, 15, 17, 19}$. (Ответ 1311)?

## Вариант 19
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(15,140,152,42)$, открытый ключ $(221, 29)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `а`: 40, `з`: 42, `м`: 64, `с`: 66. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 2050? (Ответ [3, 7, 1, 4, 5, 6, 2])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 4216753? (Ответ 2298)
1.   Вычислите НОД многочленов $-2 x^{4} - 5 x^{3} + x^{2} + 2$ и $x^{4} - x^{3} - 7 x^{2} + 8 x - 2$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + 2 x - 2$)
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x^{2} + 2$ и $x^{4} + 1$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + 2 x + 2$)
1.   Сколько чисел от $1$ до $7102$ делится хотя бы на одно из этих чисел: ${13, 15, 18, 19}$. (Ответ 1582)?

## Вариант 20
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(73,58,53,60)$, открытый ключ $(77, 29)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `а`: 26, `с`: 36, `з`: 41, `м`: 42. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 1383? (Ответ [2, 7, 4, 5, 3, 1, 6])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2145673? (Ответ 754)
1.   Вычислите НОД многочленов $-3 x^{4} - 4 x^{3} - 6 x^{2} + 4 x - 3$ и $-x^{4} + x^{3} + 3 x - 9$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + 2 x + 3$)
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + 2 x + 1$ и $2 x^{4} + 2 x^{2} + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 1$)
1.   Сколько чисел от $1$ до $2629$ делится хотя бы на одно из этих чисел: ${6, 10, 11, 15}$. (Ответ 877)?

## Вариант 21
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(6,119,81,6)$, открытый ключ $(143, 13)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `с`: 45, `о`: 62, `в`: 65, `з`: 82. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 55? (Ответ [1, 2, 5, 4, 3, 6, 7])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6314275? (Ответ 3848)
1.   Вычислите НОД многочленов $-2 x^{4} - x^{3} + 3 x^{2} - 4 x + 4$ и $-3 x^{4} - x^{3} + 6 x^{2} - 6 x + 4$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + x - 2$)
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x^{2} + 2$ и $x^{4} + 2 x^{2} + x + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + 2 x + 2$)
1.   Сколько чисел от $1$ до $9605$ делится хотя бы на одно из этих чисел: ${7, 10, 12, 17}$. (Ответ 3147)?

## Вариант 22
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(6,108,81,113)$, открытый ключ $(143, 73)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `з`: 65, `в`: 76, `с`: 83, `о`: 99. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 3615? (Ответ [6, 1, 2, 5, 4, 3, 7])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2753641? (Ответ 1404)
1.   Вычислите НОД многочленов $x^{4} - 3 x^{3} + x^{2} + 4 x - 6$ и $-3 x^{4} + 6 x^{3} + 5 x^{2} - 8 x + 3$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - x - 3$)
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{3} + 2 x^{2} + 1$ и $x^{4} + 2 x^{2} + x + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + 2 x + 2$)
1.   Сколько чисел от $1$ до $5381$ делится хотя бы на одно из этих чисел: ${6, 8, 10, 15}$. (Ответ 1792)?

## Вариант 23
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(54,45,61,54)$, открытый ключ $(65, 41)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `м`: 66, `с`: 75, `а`: 90, `з`: 97. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 759? (Ответ [2, 1, 4, 6, 5, 3, 7])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 5237461? (Ответ 3046)
1.   Вычислите НОД многочленов $2 x^{4} + 4 x^{3} - 10 x^{2} + 12 x - 6$ и $-6 x^{4} + 4 x^{3} + 2 x^{2} - 8 x + 6$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - x + 1$)
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{2} + 2$ и $x^{4} + 2 x^{2} + x + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 1$)
1.   Сколько чисел от $1$ до $3298$ делится хотя бы на одно из этих чисел: ${4, 9, 13, 14}$. (Ответ 1365)?

## Вариант 24
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(94,64,169,36)$, открытый ключ $(187, 63)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `а`: 50, `з`: 65, `с`: 77, `м`: 88. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 4937? (Ответ [7, 6, 1, 4, 5, 2, 3])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 5712364? (Ответ 3482)
1.   Вычислите НОД многочленов $4 x^{4} - 4 x^{3} + 7 x^{2} - 3 x + 2$ и $-6 x^{4} + x^{3} - 9 x^{2} - 4$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - \frac{1}{2} x + 1$)
1.   Вычислите НОД многочленов $x^{4} + x^{2} + 1$ и $2 x^{4} + x^{3} + x + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 1$)
1.   Сколько чисел от $1$ до $3929$ делится хотя бы на одно из этих чисел: ${11, 13, 15, 17}$. (Ответ 1032)?

## Вариант 25
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(80,71,74,80)$, открытый ключ $(91, 41)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `в`: 4, `с`: 39, `з`: 41, `о`: 42. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 3135? (Ответ [5, 3, 1, 6, 4, 2, 7])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6247315? (Ответ 3789)
1.   Вычислите НОД многочленов $-4 x^{4} + 4 x^{3} + 5 x^{2} - 11 x + 6$ и $-6 x^{4} + 15 x^{3} - 13 x^{2} + 3 x + 2$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - \frac{3}{2} x + 1$)
1.   Вычислите НОД многочленов $2 x^{4} + x^{2} + x + 1$ и $x^{4} + 1$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 2$)
1.   Сколько чисел от $1$ до $9272$ делится хотя бы на одно из этих чисел: ${6, 9, 13, 17}$. (Ответ 3007)?

## Вариант 26
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(24,42,18,4)$, открытый ключ $(77, 47)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `з`: 66, `о`: 69, `с`: 72, `в`: 82. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 614? (Ответ [1, 7, 2, 5, 3, 6, 4])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2513674? (Ответ 1084)
1.   Вычислите НОД многочленов $-3 x^{4} + 6 x^{3} + 6 x^{2} - 15 x + 6$ и $6 x^{4} + 15 x^{3} + 12 x^{2} - 9$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + x - 1$)
1.   Вычислите НОД многочленов $2 x^{4} + x^{3} + 2 x^{2} + 1$ и $2 x^{4} + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 2$)
1.   Сколько чисел от $1$ до $8800$ делится хотя бы на одно из этих чисел: ${13, 14, 15, 16}$. (Ответ 2166)?

## Вариант 27
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(75,69,37,75)$, открытый ключ $(77, 43)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `о`: 39, `з`: 49, `в`: 51, `с`: 59. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 96? (Ответ [1, 2, 6, 7, 5, 4, 3])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 6731524? (Ответ 4253)
1.   Вычислите НОД многочленов $6 x^{4} - x^{3} - 11 x^{2} + 4$ и $-6 x^{4} - 8 x^{3} - 7 x^{2} + x + 6$ над $\mathbf{R}[x]$. (Ответ: $x^{2} + \frac{1}{3} x - \frac{2}{3}$)
1.   Вычислите НОД многочленов $x^{4} + x^{3} + x + 1$ и $x^{4} + 2 x^{2} + 2 x + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + 2 x + 1$)
1.   Сколько чисел от $1$ до $8682$ делится хотя бы на одно из этих чисел: ${8, 9, 13, 17}$. (Ответ 2815)?

## Вариант 28
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(68,60,58,37)$, открытый ключ $(77, 7)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `м`: 39, `з`: 73, `а`: 83, `с`: 94. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 2626? (Ответ [4, 5, 7, 2, 3, 6, 1])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2174356? (Ответ 823)
1.   Вычислите НОД многочленов $-3 x^{4} + 5 x^{2} + 4 x + 1$ и $3 x^{4} - x^{3} - 4 x^{2} - 3 x - 1$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - x - 1$)
1.   Вычислите НОД многочленов $x^{4} + 2 x^{2} + x + 2$ и $2 x^{4} + x^{3} + x + 2$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 1$)
1.   Сколько чисел от $1$ до $1164$ делится хотя бы на одно из этих чисел: ${9, 10, 11, 12}$. (Ответ 365)?

## Вариант 29
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(195,63,152,195)$, открытый ключ $(221, 77)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `м`: 33, `з`: 46, `с`: 48, `а`: 62. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 3714? (Ответ [6, 1, 7, 4, 5, 3, 2])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 2754136? (Ответ 1405)
1.   Вычислите НОД многочленов $-9 x^{4} - 8 x^{2} - 4 x - 3$ и $-6 x^{4} + 7 x^{3} - 11 x^{2} + 5 x - 3$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - \frac{2}{3} x + 1$)
1.   Вычислите НОД многочленов $x^{4} + 2 x^{3} + 2 x + 1$ и $x^{4} + x^{2} + 1$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + x + 1$)
1.   Сколько чисел от $1$ до $5900$ делится хотя бы на одно из этих чисел: ${9, 15, 16, 19}$. (Ответ 1475)?

## Вариант 30
1.   Сообщение зашифровано с помощью алгоритма RSA. Для этого каждая буква была заменена
  на её порядковый номер в алфавите (счёт начинается с 2, без буквы `ё`), после чего зашифрована независимо
  от других букв.
  Шифрованое сообщение: $(145,130,128,70)$, открытый ключ $(187, 47)$. Восстановите закрытый ключ и
  расшифруйте сообщение, в ответе должно получиться русское слово из 4 букв.
1.   Определите оптимальный префиксный код для символов сообщения, с указанными частотами символов:
  `м`: 23, `а`: 69, `з`: 80, `с`: 90. При восстановлении кода, выдавайте символу с меньшей частотой — 0, а с большей — 1.
1.   Если упорядочить все перестановки чисел от 1 до 7, какая перестановка идет под номером 877? (Ответ [2, 3, 4, 6, 1, 5, 7])
1.   Если упорядочить все перестановки чисел от 1 до 7, какой по счету будет перестановка 3174526? (Ответ 1545)
1.   Вычислите НОД многочленов $-6 x^{4} + 9 x^{3} - 16 x^{2} + 11 x - 6$ и $-6 x^{4} + 3 x^{3} + 8 x^{2} - 11 x + 6$ над $\mathbf{R}[x]$. (Ответ: $x^{2} - x + \frac{2}{3}$)
1.   Вычислите НОД многочленов $2 x^{4} + 2 x^{3} + 2 x^{2} + 1$ и $x^{4} + x^{2} + 1$ над $\mathbf{Z}_3[x]$. (Ответ $x^{2} + 2 x + 1$)
1.   Сколько чисел от $1$ до $7419$ делится хотя бы на одно из этих чисел: ${2, 5, 7, 11}$. (Ответ 5107)?